In [1]:
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1011. NC Well Arsenic/P1011.2. Analyses/P1011.2.1. Data Imputation/Output')
cur_date = "020924"

library(missForest)
library(imputeLCMD)
library(readxl)
library(openxlsx)
library(tidyverse)

# reading in file
well_df = data.frame(read_excel("Input/Union_WellData_020924.xlsx", sheet = 2))

# converting some cols to numeric
numeric_df = data.frame(apply(well_df[,c(5,7,9,11,13,14,22)], 2, as.numeric))
recombined_df = cbind(well_df[,-c(5,7,9,11,13,14,22)], numeric_df)
well_df = recombined_df[,c(1:4,17,5,18,6,19,7,20,8,21,22,10,11,13,15,23,16)] 

Loading required package: tmvtnorm

Loading required package: mvtnorm

Loading required package: Matrix

Loading required package: stats4

Loading required package: gmm

Loading required package: sandwich

Loading required package: norm

This package has some major limitations
(for example, it does not work reliably when
the number of variables exceeds 30),
and has been superseded by the norm2 package.

Loading required package: pcaMethods

Loading required package: Biobase

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, aperm, append, as.data.frame, basename, cbind,
    colnames, dirname, do.call, duplicated, eval, evalq, Filter, Find,
    get, grep, grepl, intersect, is.unsorted, lapply, Map, mapply,
    match, mget, order, paste, pmax, pmax.int, pmin, pmin.int,
    Position, rank, rbind, Reduce,

Warning message:
“Expecting numeric in R2801 / R2801C18: got 'NA'”
Warning message:
“Expecting numeric in Q2802 / R2802C17: got 'NA'”
Warning message:
“Expecting numeric in R2802 / R2802C18: got 'NA'”
Warning message:
“Expecting numeric in Q2803 / R2803C17: got 'NA'”
Warning message:
“Expecting numeric in R2803 / R2803C18: got 'NA'”
Warning message:
“Expecting numeric in Q2804 / R2804C17: got 'NA'”
Warning message:
“Expecting numeric in R2804 / R2804C18: got 'NA'”
Warning message:
“Expecting numeric in Q2805 / R2805C17: got 'NA'”
Warning message:
“Expecting numeric in R2805 / R2805C18: got 'NA'”
Warning message:
“Expecting numeric in Q2806 / R2806C17: got 'NA'”
Warning message:
“Expecting numeric in R2806 / R2806C18: got 'NA'”
Warning message:
“Expecting numeric in Q2807 / R2807C17: got 'NA'”
Warning message:
“Expecting numeric in R2807 / R2807C18: got 'NA'”
Warning message:
“Expecting numeric in Q2808 / R2808C17: got 'NA'”
Warning message:
“Expecting numeric in R2808 / R2808C18: got '

Warning message:
“Expecting numeric in Q2863 / R2863C17: got 'NA'”
Warning message:
“Expecting numeric in R2863 / R2863C18: got 'NA'”
Warning message:
“Expecting numeric in Q2864 / R2864C17: got 'NA'”
Warning message:
“Expecting numeric in R2864 / R2864C18: got 'NA'”
Warning message:
“Expecting numeric in Q2865 / R2865C17: got 'NA'”
Warning message:
“Expecting numeric in R2865 / R2865C18: got 'NA'”
Warning message:
“Expecting numeric in Q2866 / R2866C17: got 'NA'”
Warning message:
“Expecting numeric in R2866 / R2866C18: got 'NA'”
Warning message:
“Expecting numeric in Q2867 / R2867C17: got 'NA'”
Warning message:
“Expecting numeric in R2867 / R2867C18: got 'NA'”
Warning message:
“Expecting numeric in Q2868 / R2868C17: got 'NA'”
Warning message:
“Expecting numeric in R2868 / R2868C18: got 'NA'”
Warning message:
“Expecting numeric in Q2869 / R2869C17: got 'NA'”
Warning message:
“Expecting numeric in R2869 / R2869C18: got 'NA'”
Warning message:
“Expecting numeric in Q2870 / R2870C17: got '

Warning message:
“Expecting numeric in R2924 / R2924C18: got 'NA'”
Warning message:
“Expecting numeric in Q2925 / R2925C17: got 'NA'”
Warning message:
“Expecting numeric in R2925 / R2925C18: got 'NA'”
Warning message:
“Expecting numeric in Q2926 / R2926C17: got 'NA'”
Warning message:
“Expecting numeric in R2926 / R2926C18: got 'NA'”
Warning message:
“Expecting numeric in Q2927 / R2927C17: got 'NA'”
Warning message:
“Expecting numeric in R2927 / R2927C18: got 'NA'”
Warning message:
“Expecting numeric in Q2928 / R2928C17: got 'NA'”
Warning message:
“Expecting numeric in R2928 / R2928C18: got 'NA'”
Warning message:
“Expecting numeric in Q2929 / R2929C17: got 'NA'”
Warning message:
“Expecting numeric in R2929 / R2929C18: got 'NA'”
Warning message:
“Expecting numeric in Q2930 / R2930C17: got 'NA'”
Warning message:
“Expecting numeric in R2930 / R2930C18: got 'NA'”
Warning message:
“Expecting numeric in Q2931 / R2931C17: got 'NA'”
Warning message:
“Expecting numeric in R2931 / R2931C18: got '

Warning message:
“Expecting numeric in Q2986 / R2986C17: got 'NA'”
Warning message:
“Expecting numeric in R2986 / R2986C18: got 'NA'”
Warning message:
“Expecting numeric in Q2987 / R2987C17: got 'NA'”
Warning message:
“Expecting numeric in R2987 / R2987C18: got 'NA'”
Warning message:
“Expecting numeric in Q2988 / R2988C17: got 'NA'”
Warning message:
“Expecting numeric in R2988 / R2988C18: got 'NA'”
Warning message:
“Expecting numeric in Q2989 / R2989C17: got 'NA'”
Warning message:
“Expecting numeric in R2989 / R2989C18: got 'NA'”
Warning message:
“Expecting numeric in Q2990 / R2990C17: got 'NA'”
Warning message:
“Expecting numeric in R2990 / R2990C18: got 'NA'”
Warning message:
“Expecting numeric in Q2991 / R2991C17: got 'NA'”
Warning message:
“Expecting numeric in R2991 / R2991C18: got 'NA'”
Warning message:
“Expecting numeric in Q2992 / R2992C17: got 'NA'”
Warning message:
“Expecting numeric in R2992 / R2992C18: got 'NA'”
Warning message:
“Expecting numeric in Q2993 / R2993C17: got '

Warning message:
“Expecting numeric in R3047 / R3047C18: got 'NA'”
Warning message:
“Expecting numeric in Q3048 / R3048C17: got 'NA'”
Warning message:
“Expecting numeric in R3048 / R3048C18: got 'NA'”
Warning message:
“Expecting numeric in Q3049 / R3049C17: got 'NA'”
Warning message:
“Expecting numeric in R3049 / R3049C18: got 'NA'”
Warning message:
“Expecting numeric in Q3050 / R3050C17: got 'NA'”
Warning message:
“Expecting numeric in R3050 / R3050C18: got 'NA'”
Warning message:
“Expecting numeric in Q3051 / R3051C17: got 'NA'”
Warning message:
“Expecting numeric in R3051 / R3051C18: got 'NA'”
Warning message:
“Expecting numeric in Q3052 / R3052C17: got 'NA'”
Warning message:
“Expecting numeric in R3052 / R3052C18: got 'NA'”
Warning message:
“Expecting numeric in Q3053 / R3053C17: got 'NA'”
Warning message:
“Expecting numeric in R3053 / R3053C18: got 'NA'”
Warning message:
“Expecting numeric in Q3054 / R3054C17: got 'NA'”
Warning message:
“Expecting numeric in R3054 / R3054C18: got '

In [2]:
head(well_df)

,Tax_ID,Health_Dept_ID,Permit_No,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Metal,Concentration,Detect_Concentration,Longitude,Latitude,Geologic_Terrane,Rock_Type,Soil_Type_Condensed,Landuse_Condensed,Elevation,Stream_Distance
,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,4006015,344,09-192,NA,97,450,35,20,7.4,Ar,NA,ND,-80.55427,34.87224,CSB,MV,H,C,675.174,654.6816
2,04009002C,344,09-192,12/7/10,97,450,35,20,7.4,Mn,NA,ND,-80.55427,34.87224,CSB,MV,H,C,678.113,654.6816
3,04009002C,344,09-192,12/7/10,97,450,35,20,7.4,Cr,NA,ND,-80.55427,34.87224,CSB,MV,H,C,678.113,654.6816
4,04009002C,1525,14-18,12/7/10,65,300,32,2,7.0,Ar,NA,ND,-80.55676,34.87109,CSB,MV,D,C,678.113,454.1912
5,04030015C,1525,14-18,8/27/12,65,300,32,2,7.0,Mn,NA,ND,-80.55676,34.87109,CSB,MV,D,C,469.708,454.1912
6,04030015C,1525,14-18,8/27/12,65,300,32,2,7.0,Cr,NA,ND,-80.55676,34.87109,CSB,MV,D,C,469.708,454.1912


Imputing missing data. Random forest will be used to impute casing depth, well depth, static water depth, flow rate, and pH. Quantile regression imputation of left-censored data (QRILC) will be used to impute each metal concentration.

Typically background filters would be implemented for all data, but only the variable `Concentration` is missing a significant portion of data.

First, we're only keeping data that has a `latitude` and `longitude`, since quite a few of the variables have a significant amount of missing data.

In [3]:
smaller_well_df = well_df[!is.na(well_df$Longitude),]
head(smaller_well_df)

,Tax_ID,Health_Dept_ID,Permit_No,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Metal,Concentration,Detect_Concentration,Longitude,Latitude,Geologic_Terrane,Rock_Type,Soil_Type_Condensed,Landuse_Condensed,Elevation,Stream_Distance
,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,4006015,344,09-192,NA,97,450,35,20,7.4,Ar,NA,ND,-80.55427,34.87224,CSB,MV,H,C,675.174,654.6816
2,04009002C,344,09-192,12/7/10,97,450,35,20,7.4,Mn,NA,ND,-80.55427,34.87224,CSB,MV,H,C,678.113,654.6816
3,04009002C,344,09-192,12/7/10,97,450,35,20,7.4,Cr,NA,ND,-80.55427,34.87224,CSB,MV,H,C,678.113,654.6816
4,04009002C,1525,14-18,12/7/10,65,300,32,2,7.0,Ar,NA,ND,-80.55676,34.87109,CSB,MV,D,C,678.113,454.1912
5,04030015C,1525,14-18,8/27/12,65,300,32,2,7.0,Mn,NA,ND,-80.55676,34.87109,CSB,MV,D,C,469.708,454.1912
6,04030015C,1525,14-18,8/27/12,65,300,32,2,7.0,Cr,NA,ND,-80.55676,34.87109,CSB,MV,D,C,469.708,454.1912


In [4]:
# original number of records
dim(well_df)

# records kept
dim(smaller_well_df)

[1] 3069   20

[1] 2145   20

In [5]:
# determining if other variables need to be imputed
dim(smaller_well_df[!is.na(smaller_well_df$Geologic_Terrane),])
dim(smaller_well_df[!is.na(smaller_well_df$Rock_Type),])
dim(smaller_well_df[!is.na(smaller_well_df$Soil_Type_Condensed),])
dim(smaller_well_df[!is.na(smaller_well_df$Landuse_Condensed),])
dim(smaller_well_df[!is.na(smaller_well_df$Stream_Distance),])
dim(smaller_well_df[!is.na(smaller_well_df$Elevation),])

[1] 2145   20

[1] 2145   20

[1] 2145   20

[1] 2145   20

[1] 2145   20

[1] 1641   20

`Elevation` will be imputed using random forest too.

In [6]:
# splitting the dataset based on the metal
preimputed_df = smaller_well_df %>%
    group_by(Metal) %>%
    group_split()

arsenic_df = preimputed_df[[1]]
chromium_df = preimputed_df[[2]]
manganese_df = preimputed_df[[3]]

# RF Imputation

In [7]:
# imputing cols 5-9 and 19 using RF
random_forest_imputation = function(dataset){
    
    imputed_RF_object = missForest(as.matrix(dataset[,c(5:9,19)]))
    imputed_RF_df = imputed_RF_object$ximp
    imputed_final_df = data.frame(cbind(dataset[,1:4], imputed_RF_df[,1:5], dataset[,10:18], imputed_RF_df[,6],
                                       dataset[,20])) 
    
    #idk why i have to do this
    colnames(imputed_final_df)[19] = c("Elevation")
    
    return(imputed_final_df)
}

# calling fn
rf_imputed_arsenic_df = random_forest_imputation(arsenic_df)
rf_imputed_chromium_df = random_forest_imputation(chromium_df)
rf_imputed_manganese_df = random_forest_imputation(manganese_df)

head(rf_imputed_arsenic_df)

,Tax_ID,Health_Dept_ID,Permit_No,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Metal,Concentration,Detect_Concentration,Longitude,Latitude,Geologic_Terrane,Rock_Type,Soil_Type_Condensed,Landuse_Condensed,Elevation,Stream_Distance
,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,4006015,344,09-192,NA,97,450,35,20.0,7.4,Ar,NA,ND,-80.55427,34.87224,CSB,MV,H,C,675.174,654.6816
2,04009002C,1525,14-18,12/7/10,65,300,32,2.0,7.0,Ar,NA,ND,-80.55676,34.87109,CSB,MV,D,C,678.113,454.1912
3,04030015C,1525,14-18,8/27/12,65,300,32,2.0,8.1,Ar,7,D,-80.55676,34.87109,CSB,MV,D,C,469.708,454.1912
4,04030015J,1525,14-18,4/5/10,65,300,32,2.0,8.1,Ar,NA,ND,-80.55676,34.87109,CSB,MV,H,F,470.293,454.1912
5,04030020H,234,09-147,10/25/10,52,125,36,20.0,7.6,Ar,NA,ND,-80.55220,34.86012,CSB,MV,H,F,470.293,918.3419
6,4030041,1515,14-04,3/2/16,47,275,34,2.5,8.2,Ar,14,D,-80.56423,34.88559,CSB,MV,H,D,470.293,512.3955


# QRILC Imputation

In [8]:
# QRILC
QRILC_imputation = function(dataset){
    # """
    # Creating a function to generate missing data
    # :param (input): df with missing data
    # :output: 1 imputed df
    # """
  
    # normalizing data since that's what the QRILC function wants
    # ended up imputing another col (health dept id), since this function needs > 1 col to impute
    QRILC_prep = dataset[,c(2,11)] %>%
         mutate_all(., function(x) log2(x)) %>%
         as.matrix()

    imputed_QRILC_object = impute.QRILC(QRILC_prep, tune.sigma = 0.1)
    QRILC_log2_df = data.frame(imputed_QRILC_object[1]) 
    
    # converting back the original scale
    QRILC_df = QRILC_log2_df %>%
        mutate_all(., function(x) 2^x)
    
    # adding back in other data
    QRILC_df = data.frame(cbind(dataset[,c(1:10,13:20)], QRILC_df[,2], dataset[,12]))
    colnames(QRILC_df)[19:20] = c("Concentration", "Detect_Concentration")


   return(QRILC_df)
}

# calling fn
imputed_arsenic_df = QRILC_imputation(rf_imputed_arsenic_df)
imputed_chromium_df = QRILC_imputation(rf_imputed_chromium_df)
imputed_manganese_df = QRILC_imputation(rf_imputed_manganese_df)

In [9]:
# combining in 1 df
imputed_df = rbind(imputed_arsenic_df, imputed_chromium_df, imputed_manganese_df)
head(imputed_df)

,Tax_ID,Health_Dept_ID,Permit_No,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Metal,Longitude,Latitude,Geologic_Terrane,Rock_Type,Soil_Type_Condensed,Landuse_Condensed,Elevation,Stream_Distance,Concentration,Detect_Concentration
,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>
1,4006015,344,09-192,NA,97,450,35,20.0,7.4,Ar,-80.55427,34.87224,CSB,MV,H,C,675.174,654.6816,2.007062,ND
2,04009002C,1525,14-18,12/7/10,65,300,32,2.0,7.0,Ar,-80.55676,34.87109,CSB,MV,D,C,678.113,454.1912,1.912321,ND
3,04030015C,1525,14-18,8/27/12,65,300,32,2.0,8.1,Ar,-80.55676,34.87109,CSB,MV,D,C,469.708,454.1912,7.000000,D
4,04030015J,1525,14-18,4/5/10,65,300,32,2.0,8.1,Ar,-80.55676,34.87109,CSB,MV,H,F,470.293,454.1912,1.153921,ND
5,04030020H,234,09-147,10/25/10,52,125,36,20.0,7.6,Ar,-80.55220,34.86012,CSB,MV,H,F,470.293,918.3419,1.675360,ND
6,4030041,1515,14-04,3/2/16,47,275,34,2.5,8.2,Ar,-80.56423,34.88559,CSB,MV,H,D,470.293,512.3955,14.000000,D


In [10]:
# exporting results
write.xlsx(imputed_df, paste0(Output,"/", "Imputed_Well_Data_", cur_date, ".xlsx"), rowNames = FALSE)